### Import the needed packages

In [2]:
# package for using the the custom scripts 
import sys
sys.path.insert(0, "./../")
# package for handling files and directories
import os

import wandb
# a custom package for handling the data
import utilities as utils

from huggingface_hub import HfApi

from dotenv import load_dotenv

load_dotenv()

True

### Global variable

In [ ]:
path_full = os.environ.get("ORI_PATH")
path_dataset_ori = os.path.join(path_full, "datasets", "splitted", "original")
path_dataset_aug = os.path.join(path_full, "datasets", "splitted", "augmented")
path_model = os.path.join(path_full, "data", "model")
wandb_init_name = "implement weight training"
hf_commit_message = "implement weight training"
hyperparam = {
    "batch_size": 16,
    "img_size": 128,
    "shuffle": True,
    "model_type": "U-net",
    "metrics": ["accuracy", "MeanIoU", "Precision", "Recall"],
    "split_ratio": [7,1,2],
    "dataset_balance": True,
    "weighted_training": True,
}
ori_path = {
    "train": os.path.join(path_dataset_ori, "train"),
    "val": os.path.join(path_dataset_ori, "val"),
    "test": os.path.join(path_dataset_ori, "test"),
}
aug_path = {
    "train": os.path.join(path_dataset_aug, "train"),
    "val": os.path.join(path_dataset_aug, "val"),
    "test": os.path.join(path_dataset_aug, "test"),
}

### Import the images and mask

In [4]:
# get the original dataset
train_ori_img, train_ori_mask = utils.load_img_mask(ori_path["train"])
val_ori_img, val_ori_mask = utils.load_img_mask(ori_path["val"])
test_ori_img, test_ori_mask = utils.load_img_mask(ori_path["test"])

# get the augmented dataset
train_aug_img, train_aug_mask = utils.load_img_mask(aug_path["train"])
val_aug_img, val_aug_mask = utils.load_img_mask(aug_path["val"])
test_aug_img, test_aug_mask = utils.load_img_mask(aug_path["test"])

### Create the batched dataset

In [5]:
# create the tensor dataset for original dataset
trainset_ori = utils.create_dataset(train_ori_img, train_ori_mask, hyperparam["img_size"], hyperparam["batch_size"])
valset_ori = utils.create_dataset(val_ori_img, val_ori_mask, hyperparam["img_size"], hyperparam["batch_size"])
testset_ori = utils.create_dataset(test_ori_img, test_ori_mask, hyperparam["img_size"], hyperparam["batch_size"])

# create the tensor dataset for augmented dataset
trainset_aug = utils.create_dataset(train_aug_img, train_aug_mask, hyperparam["img_size"], hyperparam["batch_size"])
valset_aug = utils.create_dataset(val_aug_img, val_aug_mask, hyperparam["img_size"], hyperparam["batch_size"])
testset_aug = utils.create_dataset(test_aug_img, test_aug_mask, hyperparam["img_size"], hyperparam["batch_size"])

### Calculate label weight

In [6]:
train_weight = utils.calculate_weight(trainset_ori)
train_weight

{0: 0.3392, 1: 90.8128, 2: 32.1602}

### Train model

In [ ]:
# create the unet model
unet_custom = utils.custom_unet()
file_model_name = "unet_model_ori"
# train the model using original dataset
unet_custom, history, ev_result = utils.train_model(unet_custom, trainset_ori, valset_ori, testset_ori, path_model, file_model_name, train_weight)
print(f"test loss: {ev_result[0]}", f"test accuracy: {ev_result[1]}",
        f"test mean IoU: {ev_result[2]}", f"test precision: {ev_result[3]}",
        f"test recall: {ev_result[4]}", sep="\n")

Epoch 1/10
16/16 [==============================] - 29s 452ms/step - loss: 1.1345 - accuracy: 5.4584e-06 - mean_iou: 0.3333 - precision: 0.6405 - recall: 0.1870 - val_loss: 0.9821 - val_accuracy: 0.0000e+00 - val_mean_iou: 0.3333 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/10
16/16 [==============================] - 8s 121ms/step - loss: 0.9097 - accuracy: 0.0000e+00 - mean_iou: 0.3333 - precision: 0.7682 - recall: 0.0568 - val_loss: 0.8731 - val_accuracy: 0.0000e+00 - val_mean_iou: 0.3333 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 3/10
16/16 [==============================] - 7s 148ms/step - loss: 0.7791 - accuracy: 0.0000e+00 - mean_iou: 0.3333 - precision: 0.9286 - recall: 0.1480 - val_loss: 0.7327 - val_accuracy: 0.0000e+00 - val_mean_iou: 0.3333 - val_precision: 0.0881 - val_recall: 2.3736e-05
Epoch 4/10
16/16 [==============================] - 8s 132ms/step - loss: 0.6836 - accuracy: 0.0000e+00 - mean_iou: 0.3333 - precision: 0.9883 - recall: 0.73

In [ ]:
api = HfApi()
repo = api.create_repo("CAD-Glaucoma_Segmentation", repo_type="model", exist_ok=True)
api.upload_file(
    repo_id=repo.repo_id,
    path_or_fileobj=os.path.join("./../../../data/model", f"{file_model_name}.h5"),
    path_in_repo=f"model/{file_model_name}.h5",
    repo_type=repo.repo_type,
    commit_message=hf_commit_message,
)

In [ ]:
hyperparam["aug"] = False
wandb.init(project="CAD - Glaucoma Segmentation",
            name=f"ori - {wandb_init_name}",
            config=hyperparam)
for epoch in range(len(history.history["loss"])):
    wandb.log({
        "train_loss": history.history["loss"][epoch],
        "train_accuracy": history.history["accuracy"][epoch],
        "train_meanIoU": history.history["mean_iou"][epoch],
        "train_precision": history.history["precision"][epoch],
        "train_recall": history.history["recall"][epoch],
        "val_loss": history.history["val_loss"][epoch],
        "val_accuracy": history.history["val_accuracy"][epoch],
        "val_meanIoU": history.history["val_mean_iou"][epoch],
        "val_precision": history.history["val_precision"][epoch],
        "val_recall": history.history["val_recall"][epoch],
    })
wandb.log_artifact(os.path.join("./../../../data/model", f"{file_model_name}.h5"))
wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: bugi-sulistiyo (bugi-sulistiyo-universitas-mulawarman). Use `wandb login --relogin` to force relogin


train_accuracy,▃▁▁▁▁▁▁▁▃█
train_loss,█▆▅▄▄▃▂▂▁▁
train_meanIoU,█▁▁▁▁▁▁▁▁▁
train_precision,▁▄▇███████
train_recall,▂▁▂▆██████
val_accuracy,▁▁▁▁▁▁▁▁▁▁
val_loss,█▇▆▅▄▃▂▂▁▁
val_meanIoU,▁▁▁▁▁▁▁▁▁▁
val_precision,▁▁▂███████
val_recall,▁▁▁███████
train_accuracy,2e-05


unet_model_ori.h5: 100%|██████████| 1.57M/1.57M [00:01<00:00, 831kB/s] 


CommitInfo(commit_url='https://huggingface.co/bugi-sulistiyo/CAD-Glaucoma_Segmentation/commit/b777f22e311a5ad4b3fb2eaad6af65801d861f3b', commit_message='initial upload the original model', commit_description='', oid='b777f22e311a5ad4b3fb2eaad6af65801d861f3b', pr_url=None, repo_url=RepoUrl('https://huggingface.co/bugi-sulistiyo/CAD-Glaucoma_Segmentation', endpoint='https://huggingface.co', repo_type='model', repo_id='bugi-sulistiyo/CAD-Glaucoma_Segmentation'), pr_revision=None, pr_num=None)

In [8]:
# create the unet model
unet_custom = utils.custom_unet()
file_model_name = "unet_model_aug"
# train the model using augmented dataset
unet_custom, history, ev_result = utils.train_model(unet_custom, trainset_aug, valset_aug, testset_aug, file_model_name)
print(f"test loss: {ev_result[0]}", f"test accuracy: {ev_result[1]}",
        f"test mean IoU: {ev_result[2]}", f"test precision: {ev_result[3]}",
        f"test recall: {ev_result[4]}", sep="\n")

Epoch 1/10
16/16 [==============================] - 9s 158ms/step - loss: 1.1135 - accuracy: 0.0000e+00 - mean_iou: 0.3333 - precision: 0.6439 - recall: 0.2742 - val_loss: 1.0014 - val_accuracy: 0.0000e+00 - val_mean_iou: 0.3333 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00
Epoch 2/10
16/16 [==============================] - 5s 122ms/step - loss: 0.7741 - accuracy: 2.6465e-06 - mean_iou: 0.3333 - precision: 0.9602 - recall: 0.5302 - val_loss: 0.8339 - val_accuracy: 0.0000e+00 - val_mean_iou: 0.3333 - val_precision: 1.0000 - val_recall: 6.1035e-05
Epoch 3/10
16/16 [==============================] - 4s 97ms/step - loss: 0.6475 - accuracy: 0.0000e+00 - mean_iou: 0.3333 - precision: 0.9858 - recall: 0.8852 - val_loss: 0.6746 - val_accuracy: 0.0000e+00 - val_mean_iou: 0.3333 - val_precision: 0.9826 - val_recall: 0.9808
Epoch 4/10
16/16 [==============================] - 4s 101ms/step - loss: 0.5624 - accuracy: 0.0000e+00 - mean_iou: 0.3333 - precision: 0.9905 - recall: 0.9835 - val_l

In [9]:
hyperparam["aug"] = True
wandb.init(project="CAD - Glaucoma Segmentation",
            name=f"aug - {wandb_init_name}",
            config=hyperparam)
for epoch in range(len(history.history["loss"])):
    wandb.log({
        "train_loss": history.history["loss"][epoch],
        "train_accuracy": history.history["accuracy"][epoch],
        "train_meanIoU": history.history["mean_iou"][epoch],
        "train_precision": history.history["precision"][epoch],
        "train_recall": history.history["recall"][epoch],
        "val_loss": history.history["val_loss"][epoch],
        "val_accuracy": history.history["val_accuracy"][epoch],
        "val_meanIoU": history.history["val_mean_iou"][epoch],
        "val_precision": history.history["val_precision"][epoch],
        "val_recall": history.history["val_recall"][epoch],
    })
wandb.log_artifact(os.path.join("./../../../data/model", f"{file_model_name}.h5"))
wandb.finish()
api.upload_file(
    repo_id=repo.repo_id,
    path_or_fileobj=os.path.join("./../../../data/model", f"{file_model_name}.h5"),
    path_in_repo=f"model/{file_model_name}.h5",
    repo_type=repo.repo_type,
    commit_message="initial upload the aug model",
)

train_accuracy,▁▁▁▁▁▁▁▃▆█
train_loss,█▅▄▄▃▃▂▂▁▁
train_meanIoU,▁█████████
train_precision,▁▇████████
train_recall,▁▃▇███████
val_accuracy,▁▁▁▁▁▁▁▁▁▁
val_loss,█▇▅▄▃▃▂▂▁▁
val_meanIoU,▁▁▁▁▁▁▁▁▁▁
val_precision,▁█████████
val_recall,▁▁████████
train_accuracy,0.00041


unet_model_aug.h5: 100%|██████████| 1.57M/1.57M [00:01<00:00, 961kB/s] 


CommitInfo(commit_url='https://huggingface.co/bugi-sulistiyo/CAD-Glaucoma_Segmentation/commit/7ee236598a20f859e7470f1a6af3c2d3138c07de', commit_message='initial upload the aug model', commit_description='', oid='7ee236598a20f859e7470f1a6af3c2d3138c07de', pr_url=None, repo_url=RepoUrl('https://huggingface.co/bugi-sulistiyo/CAD-Glaucoma_Segmentation', endpoint='https://huggingface.co', repo_type='model', repo_id='bugi-sulistiyo/CAD-Glaucoma_Segmentation'), pr_revision=None, pr_num=None)